Kaggle: https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/notebooks

In [ ]:
import numpy as np
import pandas as pd
import os
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import os
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
import zipfile

with zipfile.ZipFile("../input/train.zip","r") as z:
    z.extractall(".")
with zipfile.ZipFile("../input/test.zip","r") as z:
    z.extractall(".")

In [ ]:
TRAIN_DIR = '/kaggle/working/train/'
TEST_DIR = '/kaggle/working/test/'

# Explantory Data Analyisis

In [ ]:
import cv2

In [ ]:
training_files = os.listdir(TRAIN_DIR)

In [ ]:
training_files[0]

In [ ]:
cv2.imread(TRAIN_DIR + training_files[0]).shape

In [ ]:
test_files = os.listdir(TEST_DIR)

In [ ]:
test_files[0]

# Prepare Data

In [ ]:
IMG_SIZE = 50

In [ ]:
def create_dataset(folder_path, flag_training=True):
    def identify_label(file_name):
        dog_or_cat = file_name.split('.')[0]
        if dog_or_cat == 'cat':
            return np.array([1, 0])
        elif dog_or_cat == 'dog':
            return np.array([0, 1])
        
    def identify_image_id(file_name):
        return file_name.split('.')[0]
    
    result = []
    for file_name in tqdm(os.listdir(folder_path)):
        file_path = os.path.join(folder_path, file_name)
        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        if flag_training:
            img_data = np.array(img)
            label = identify_label(file_name)
            result.append([img_data, label])
        else:
            img_data = np.array(img)
            img_id = identify_image_id(file_name)
            result.append([img_data, img_id])
    return result

In [ ]:
train_dataset = create_dataset(TRAIN_DIR)
test_dataset = create_dataset(TEST_DIR, flag_training=False)

# CNN Model

1.Model Definition

In [ ]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [ ]:
FILTER_SIZE = 10
LR = 1e-3
EPOCH = 3

In [ ]:
class CNN:
    def __init__(self):
        self.model = self.__build_cnn()
    
    def fit(self, train_dataset):
        x_train, y_train, x_val, y_val = self.__load_training_validation(train_dataset)
        self.model.fit({'input': x_train}, {'targets': y_train}, \
                  validation_set=({'input': x_val}, {'targets': y_val}), \
                  n_epoch=EPOCH, snapshot_step=500, show_metric=True, run_id='model' )
        
    def predict(self, input_data_list):
        return self.model.predict(input_data_list)
               
    def __build_cnn(self):
        # Note: Input
        cnn = input_data(shape = [None, IMG_SIZE, IMG_SIZE, 1], name = 'input')
        
        # Note:conv_2d - http://tflearn.org/layers/conv/
        #       The default padding will make output image size the same.
        #       The default step size is 1.
        #       max_pool_2d - http://tflearn.org/layers/conv/#max-pooling-2d
        
        # Note: Layer 1 CNN with 32 filters and 5*5 filter size. 
        #       Dimension: (?, 50, 50, 1) -> (?, 50, 50, 32) -> (?, 10, 10, 32)
        cnn = conv_2d(cnn, 32, FILTER_SIZE, activation='relu')
        cnn = max_pool_2d(cnn, FILTER_SIZE)
        # Note: Layer 2 CNN with 64 filters and 5*5 filter size.
        #       Dimension: (?, 10, 10, 32) -> (?, 10, 10, 64) -> (?, 2, 2, 64)
        cnn = conv_2d(cnn, 64, FILTER_SIZE, activation='relu')
        cnn = max_pool_2d(cnn, FILTER_SIZE)
        # Note: Layer 3 CNN with 128 filters and 5*5 filter size.
        #       Dimension: (?, 2, 2, 64) -> (?, 2, 2, 128) -> (?, 1, 1, 128)
        cnn = conv_2d(cnn, 128, FILTER_SIZE, activation='relu')
        cnn = max_pool_2d(cnn, FILTER_SIZE)      
        # Note: Layer 4 CNN with 64 filters and 5*5 filter size.
        #       Output: (?, 1, 1, 128) -> (?, 1, 1, 64) -> (?, 1, 1, 64)
        cnn = conv_2d(cnn, 64, FILTER_SIZE, activation='relu')
        cnn = max_pool_2d(cnn, FILTER_SIZE)               
        # Note: Layer 5 CNN with 32 filters and 5*5 filter size.
        #       Output: (?, 1, 1, 64) -> (?, 1, 1, 32) -> (?, 1, 1, 32)
        cnn = conv_2d(cnn, 32, FILTER_SIZE, activation='relu')
        cnn = max_pool_2d(cnn, FILTER_SIZE)
        
        # Note: Layer 6 Fully Connected
        #       Output: (?, 1, 1, 32) -> (?, 1024)
        cnn = fully_connected(cnn, 1024, activation = 'relu')
        # Note: Layer 7 Dropout
        cnn = dropout(cnn, 0.8)
        # Note: Layer 8 Fully Connected
        #       Output: (?, 1024) -> (?, 2)
        cnn = fully_connected(cnn, 2, activation='softmax')
        
        # Note: Output
        cnn = regression(cnn, optimizer='adam', learning_rate = LR, \
                         loss='categorical_crossentropy', name='targets')
        model = tflearn.DNN(cnn, tensorboard_dir='log')
        
        return model
        
    def __load_training_validation(self, train_dataset, validation_portion=0.2):
        num_of_obs = len(train_dataset)
        cutoff = int(num_of_obs * (1 - validation_portion))
        train, validation = train_dataset[:cutoff], train_dataset[cutoff:]
        
        x_train = np.array([i[0] for i in train], dtype=np.float64).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        y_train = np.array([i[1] for i in train], dtype=np.float64)
        x_val = np.array([i[0] for i in validation], dtype=np.float64).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        y_val = np.array([i[1] for i in validation], dtype=np.float64)
        return x_train, y_train, x_val, y_val

2.Train

In [ ]:
cnn = CNN()
cnn.fit(train_dataset)

3.Predict

In [ ]:
def predict_some_test_samples_and_show(test_dataset):
    fig = plt.figure()
    for num, data in enumerate(test_dataset[:16]):
        img_data, img_num = data[0], data[1]
        img_data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
        pred_prob = cnn.predict([img_data])[0]
        pred_label = 'Dog' if np.argmax(pred_prob) == 1 else 'Cat'
        
        subplot = fig.add_subplot(4, 4, num+1)
        subplot.imshow(data[0], cmap='gray')
        plt.title(pred_label)
        subplot.axes.get_xaxis().set_visible(False)
        subplot.axes.get_yaxis().set_visible(False)
    
    plt.tight_layout()
    plt.show()

In [ ]:
def predict_for_all_examples(test_dataset):
    img_ids, img_labels = [], []
    for sample in tqdm(test_dataset):
        img_data, img_id = sample[0], sample[1]
        img_data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
        pred_prob = cnn.predict([img_data])[0]
        pred_label = 1 if np.argmax(pred_prob) == 1 else 0 # Note: 1=dog, 0=cat
        img_ids.append(img_id)
        img_labels.append(pred_label)
    return pd.DataFrame({'id': img_ids, 'label': img_labels})

In [ ]:
predict_some_test_samples_and_show(test_dataset)

In [ ]:
submission = predict_for_all_examples(test_dataset)
submission.to_csv('submission.csv', index=False,header=True)